# 基板の設計
キーボードの基板を設計します。

通常の工程で基板を作るにはまず`回路図エディタ`で`回路図`を書いて使用する部品とその結線情報をまとめて`ネットリスト`として出力し、`基板エディタ`を用いて実際に`基板`へ部品を配置・配線します。

# 記事を読むための注意
このノートブックには2回実行すると正常に動作しないセルが含まれています。それを防止するために上のメニューから`ランタイム>全てのセルを実行`してから読み始めると良いかもしれません。(実行完了には少し時間がかかります)

# skidlでネットリスト(回路図)を設計する
Pythonを使った設計では回路図を書かずにコードで使用する部品と結線情報を定義してネットリストを出力します。

[skidl](https://github.com/devbisme/skidl)というライブラリを用います。このライブラリは独自フォーマットの部品ライブラリだけでなく、kicadの部品ライブラリをインポートして使用します。また、ネットリストはkicadのPCBエディタと互換性があり直接kicadへ読み込んで回路図を書くこともできます。

- skidl  https://github.com/devbisme/skidl
- skidlのドキュメント  https://devbisme.github.io/skidl/

## kicadライブラリのダウンロード
以下のセルを実行してkicadのライブラリが入ったリポジトリをクローンしておきます。

In [1]:
!git clone https://github.com/hsgw/keyboard-made-by-python/

Cloning into 'keyboard-made-by-python'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 64 (delta 13), reused 57 (delta 6), pack-reused 0
Unpacking objects: 100% (64/64), done.


## skidlのインストールとインポート
ノートブック上に[skidl](https://github.com/devbisme/skidl)をインストールしてインポートしておきます。
インストールには少し時間がかかるかもしれません。

このとき、インストールとインポートは別のセルにしておくと何度もインストールしなくて済むのでセルを分けておきます。

In [5]:
!pip install skidl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from skidl import *
# kicadのデフォルトパスが環境変数に設定されていないのでWarningが出ます

## 定数を宣言する
何度も出てくる値を定数として宣言しておきます。この値を使い回すことで変更に強い設計になります。


In [4]:
KEY_COUNT = 10
COL_COUNT = 4
ROW_COUNT = 3
MATRIX_MAP = [
        (0,1),(0,2),(0,3),  
        (1,1),(1,2),(1,3),
  (2,0),(2,1),(2,2),(2,3)
]

## kicadライブラリの読み込みと部品の準備
使用するkicadライブラリのパスを追加をします。

kicadがインストールされている環境であれば通常のライブラリへのパスがデフォルトで追加されているはずですが、今回はkicadのインストールされていない環境で実行するために使う部品だけ別に用意しておきます。
先程githubからcloneしてきた`keyboard-made-by-python/hardware/kicad_libs`に入っています。

シンボルとフットプリントを結びつけて必要な個数だけ部品を用意します。

In [5]:
# 使用するライブラリを登録する
# シンボル・フットプリントどちらも別で登録する
lib_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs")
footprint_search_paths[KICAD].append("keyboard-made-by-python/hardware/kicad_libs/kicad.pretty")

# 同じ部品を何回も使うならシンボルとフットプリントを結びつけたテンプレートとして読み込んでおく
diode = Part(
  "kicad_symbols", "D_Small_ALT", TEMPLATE, footprint="kicad:D_SOD123_hand"
)
switch = Part(
  "kicad_symbols",
  "SW_Push",
  TEMPLATE,
  footprint="kicad:SW_Cherry_MX_1.00u_PCB",
)

# 宣言した定数とテンプレートを使ってダイオードとスイッチを用意する
# それぞれ KEY_COUNT 個用意して配列にいれておく
diodes = diode(KEY_COUNT)
switches = switch(KEY_COUNT)

# ひとつしか使わない部品はそのまま読み込む
xiao = Part("kicad_symbols", "xiao_rp2040", footprint="kicad:xiao_rp2040")
oled = Part("kicad_symbols", "oled_i2c", footprint="kicad:oled_i2c")

# printするとシンボルのPin情報が見えます
print(diode, switch, xiao, oled)


 D_Small_ALT (): Diode, small symbol, filled shape
    Pin None/1/K/PASSIVE
    Pin None/2/A/PASSIVE 
 SW_Push (): Push button switch, generic, two pins
    Pin None/1/1/PASSIVE
    Pin None/2/2/PASSIVE 
 xiao_rp2040 (): 
    Pin U1/1/P26/ADC0/I2C1_SDA/BIDIRECTIONAL
    Pin U1/10/UART1_TX/I2C0_SDA/SPI0_RX/P4/BIDIRECTIONAL
    Pin U1/11/I2C1_SCL/SPI0_TX/P3/BIDIRECTIONAL
    Pin U1/12/3.3V/POWER-OUT
    Pin U1/13/GND/POWER-OUT
    Pin U1/14/5V/POWER-OUT
    Pin U1/2/P27/ADC1/I2C1_SCL/BIDIRECTIONAL
    Pin U1/3/P28/ADC2/BIDIRECTIONAL
    Pin U1/4/P29/ADC3/BIDIRECTIONAL
    Pin U1/5/P6/I2C1_SDA/SPI0_SCK/BIDIRECTIONAL
    Pin U1/6/P7/I2C1_SCL/SPI0_TX/BIDIRECTIONAL
    Pin U1/7/P0/UART0_TX/I2C0_SDA/SPI0_RX/BIDIRECTIONAL
    Pin U1/8/UART0_RX/I2C0_SCL/SPI0_CSn/P1/BIDIRECTIONAL
    Pin U1/9/I2C1_SDA/SPI0_SCK/P2/BIDIRECTIONAL 
 oled_i2c (): 
    Pin DISP1/1/SDA/BIDIRECTIONAL
    Pin DISP1/2/SCL/INPUT
    Pin DISP1/3/Vcc/POWER-IN
    Pin DISP1/4/GND/POWER-IN


## PinやNetを接続する
用意した部品のピン同士を接続を入力して回路の構成を定義します。

このときなるべく配線自体に`Net`名をつけるようにします。`Net`を基準に`Pin`を繋いでいくことで同じところにつながるもの(電源やスイッチマトリクス)を整理して見やすいコードにします。また、kicadへネットリストをインポートしたときに表示されるので目印にもなります。

今回の設計でも基板配線時に`ピン番号`を`Net`から読み出します。

In [6]:
# Pin情報を表示しながら配線すると便利
print(xiao)


 xiao_rp2040 (): 
    Pin U1/1/P26/ADC0/I2C1_SDA/BIDIRECTIONAL
    Pin U1/10/UART1_TX/I2C0_SDA/SPI0_RX/P4/BIDIRECTIONAL
    Pin U1/11/I2C1_SCL/SPI0_TX/P3/BIDIRECTIONAL
    Pin U1/12/3.3V/POWER-OUT
    Pin U1/13/GND/POWER-OUT
    Pin U1/14/5V/POWER-OUT
    Pin U1/2/P27/ADC1/I2C1_SCL/BIDIRECTIONAL
    Pin U1/3/P28/ADC2/BIDIRECTIONAL
    Pin U1/4/P29/ADC3/BIDIRECTIONAL
    Pin U1/5/P6/I2C1_SDA/SPI0_SCK/BIDIRECTIONAL
    Pin U1/6/P7/I2C1_SCL/SPI0_TX/BIDIRECTIONAL
    Pin U1/7/P0/UART0_TX/I2C0_SDA/SPI0_RX/BIDIRECTIONAL
    Pin U1/8/UART0_RX/I2C0_SCL/SPI0_CSn/P1/BIDIRECTIONAL
    Pin U1/9/I2C1_SDA/SPI0_SCK/P2/BIDIRECTIONAL


In [7]:
# スイッチマトリクスのROW,COLのネットリストが入った配列を作る
netRows = [Net(f"ROW{i}") for i in range(ROW_COUNT)]
netCols = [Net(f"COL{i}") for i in range(COL_COUNT)]


# ROWのNet -> スイッチの1ピン  スイッチの2ピン -> ダイオードのカソード  ダイオードのアノード -> COLのNet をまとめて宣言する
# NetやPinは`&`で繋ぐと接続される
# 部品のPinには part["pin name"]でアクセス出来る
# 部品のPinの添字を2つにするとinとoutになる
# 例: sw["1"]につながるNetかPin & sw["1 2"] & sw["2"]に繋がるNetかPin
for sw, d, mapping in zip(switches, diodes, MATRIX_MAP):
  netRows[mapping[0]] & sw["1 2"] & d["K A"] & netCols[mapping[1]]

# スイッチマトリクスとxiaoを接続する
# NetにPinを`+`ことで接続する
netCols[0] += xiao[8]
netCols[1] += xiao[3]
netCols[2] += xiao[4]
netCols[3] += xiao[5]

netRows[0] += xiao[1]
netRows[1] += xiao[6]
netRows[2] += xiao[7]

# oledとxiaoも接続する
# Net("3.3V")みたいに直接Netを宣言して繋いでもOK
Net("3.3V") & oled["Vcc"] & xiao["3.3V"]
Net("GND") & oled["GND"] & xiao["GND"]
Net("SDA") & oled["SDA"] & xiao[9]
Net("SCL") & oled["SCL"] & xiao[11]

# printすると接続されているPinが表示される
print(netRows[0])

ROW0: Pin SW1/1/1/PASSIVE, Pin SW2/1/1/PASSIVE, Pin SW3/1/1/PASSIVE, Pin U1/1/P26/ADC0/I2C1_SDA/BIDIRECTIONAL


## ERCとネットリストの出力
ERC(Electrical Rule Check・回路図のルールチェック)をかけて、ネットリストを出力します。
これで回路図・ネットリストは完成です！

In [8]:
# 他のセルを複数回実行しているとエラーが出たり回路が複数個になったりするかもしれません
# そのときは`メニュー`の`ランタイム -> ランタイムを再起動`して最初からやりなおすか`再起動してすべてのセルを実行`してください
# 未結線のWarningが出ますが問題ありません
ERC()
generate_netlist(file_="keyboard.net")

ERC WARNING: Unconnected pin: BIDIRECTIONAL pin 10/UART1_TX/I2C0_SDA/SPI0_RX/P4 of xiao_rp2040/U1.
ERC WARNING: Unconnected pin: POWER-OUT pin 14/5V of xiao_rp2040/U1.
ERC WARNING: Unconnected pin: BIDIRECTIONAL pin 2/P27/ADC1/I2C1_SCL of xiao_rp2040/U1.

3 warnings found while running ERC.
0 errors found while running ERC.


No errors or warnings found while generating netlist.



'(export (version D)\n  (design\n    (source "/content/skidl")\n    (date "12/17/2022 05:29 PM")\n    (tool "SKiDL (1.1.0)"))\n  (components\n    (comp (ref D1)\n      (value D_Small_ALT)\n      (footprint kicad:D_SOD123_hand)\n      (fields\n        (field (name F0) D)\n        (field (name F1) D_Small_ALT)\n        (field (name F2) kicad:D_SOD123_hand))\n      (libsource (lib kicad_symbols) (part D_Small_ALT))\n      (sheetpath (names /top/2031437178528088810) (tstamps /top/2031437178528088810)))\n    (comp (ref D10)\n      (value D_Small_ALT)\n      (footprint kicad:D_SOD123_hand)\n      (fields\n        (field (name F0) D)\n        (field (name F1) D_Small_ALT)\n        (field (name F2) kicad:D_SOD123_hand))\n      (libsource (lib kicad_symbols) (part D_Small_ALT))\n      (sheetpath (names /top/11538802863818834611) (tstamps /top/11538802863818834611)))\n    (comp (ref D2)\n      (value D_Small_ALT)\n      (footprint kicad:D_SOD123_hand)\n      (fields\n        (field (name F0) D)\

出力されたネットリストは今記事では使用しませんが、kicadのpcbnewのネットリスト読み込みからインポートしてそのまま基板を作ることも出来ます。

![pcbnewへネットリストをインポートした](https://github.com/hsgw/keyboard-made-by-python/blob/main/notebook/imgs/kicad_pcbnew.png?raw=1)

# pcbflowで基板を設計する
skidlで設計した結線情報を活用しながら[pcbflow](https://github.com/michaelgale/pcbflow)で実際の基板に部品を配置して配線をします。

pcbflowは開発中(?)のようで意図しない動作や不具合があったため修正と追加をしました。今記事ではその[folk](https://github.com/hsgw/pcbflow/tree/fix_kicad)を使用します。

- pcbflow https://github.com/michaelgale/pcbflow
- pcvflowのドキュメントはREADME.mdにあります
- folkして修正したpcbflow https://github.com/hsgw/pcbflow/tree/fix_kicad

## pcbflowのダウンロードとインストール
以下のセルを実行してインストールします。
インポートしてエラーがないことを確認します。

In [ ]:
!git clone https://github.com/hsgw/pcbflow/ -b fix_kicad
!cd pcbflow && python setup.py install

# from stable defusion
# ランタイムを再起動します
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

fatal: destination path 'pcbflow' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing pcbflow.egg-info/PKG-INFO
writing dependency_links to pcbflow.egg-info/dependency_links.txt
writing requirements to pcbflow.egg-info/requires.txt
writing top-level names to pcbflow.egg-info/top_level.txt
adding license file 'LICENSE.md'
writing manifest file 'pcbflow.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pcbflow
copying build/lib/pcbflow/util.py -> build/bdist.linux-x86_64/egg/pcbflow
copying build/lib/pcbflow/__init__.py -> build/bdist.linux-x86_64/egg/pcbflow
copying build/lib/pcbflow/part.py -> build/bdist.linux-x86_64/egg/pcbflow
copying build/lib/pcbflow/route.py -> build/bdist.linux-x86_64/egg/pcbflow
copying build/lib/pcbflow/gerber.py -> build/bdist.linux-x86_64/egg/pcbflow
creating build/bd

In [2]:
from pcbflow import *

## 定数を宣言する
ネットリストと同じようによく使う値を定数として宣言しておきます。   
座標のY軸が反転していて違和感があったので変換するための関数も宣言しています。

In [3]:
BOARD_WIDTH = 76.0
BOARD_HEIGHT = 57.0

KEY_PITCH = 19.0

SCREW_HOLE = 2.2

LAYER_TOP = "GTL"
LAYER_BOTTOM = "GBL"

# Y軸の座標を反転させる関数
def pos(x, y):
  return (x, BOARD_HEIGHT - y)

## 基板と回路の変数を宣言する
基板と回路の変数を宣言しておきます。
基板はpcbflowのもの、回路はskidlのものです。

回路から部品や配線情報を得ます。

In [4]:
# skidlで作った回路情報
circuit = builtins.default_circuit
# pcbflowの基板
board = Board((BOARD_WIDTH, BOARD_HEIGHT))

NameError: ignored

## デザインルールを設定する
ドリルの大きさや配線の太さなど、基板のデザインルールを設定します。